In [ ]:
import os
import sys
import glob
import pandas as pd
import numpy as np
import xarray as xr
import collections
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.ndimage import gaussian_filter
from importlib import reload
import datetime as dt
# sys.path.append('/home/gfat/software')
sys.path.append(r'C:\Users\Usuario\Documents\gitlab\utils_gfat')
import lidar
import helper_functions
import lidarQA
import pdb


In [ ]:
datestr, hour_ini, hour_end = '20210624', '223000', '233000' 
date_ini = dt.datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
ini_date = dt.datetime.strptime(ini_date_str,'%Y%m%dT%H%M%S')
end_date = dt.datetime.strptime(end_date_str,'%Y%m%dT%H%M%S')

In [ ]:
output_dir = r'C:\Users\Usuario\Downloads\geomet_output'

In [ ]:
path1a = r'C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a'
rs_fl = os.path.join(path1a, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Prs_rs_xf_%s.nc' % datestr)
rs = xr.open_dataset(rs_fl)


In [ ]:
dc_fl = os.path.join(path1a, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Pdc_rs_xf_20210624_0444.nc')
dc = xr.open_dataset(dc_fl)

In [ ]:
rs = rs.sel(time=slice(ini_date_str, end_date_str))
dc = dc.mean('time')

In [ ]:
type(rs['signal_532xpa'].values)

In [ ]:
rs['signal_532xpa'].values.shape,  dc['signal_532xpa'].values.shape

In [ ]:
rs['signal_532xpa'].values - dc['signal_532xpa'].values

In [ ]:
dc_rs_np = np.nanmean(rs['signal_532xpa'].values - dc['signal_532xpa'].values, axis=0)
bg = dc_rs.sel(range=slice(75000,105000)).mean('range').values
bg_dc_rs_np = dc_rs_np - bg
plt.plot(rs.range.values, dc_rs_np)
plt.plot(rs.range.values, bg_dc_rs_np)

In [ ]:
(rs['signal_532xpa'] - dc['signal_532xpa']).mean('time').plot()

In [ ]:
rs['signal_532xpa'].mean('time').plot()
dc['signal_532xpa'].plot()
dc_rs = (rs['signal_532xpa'].mean('time') -dc['signal_532xpa'])
dc_rs.plot()
bg = dc_rs.sel(range=slice(75000,105000)).mean('range')
bg_dc_rs = dc_rs - bg
bg_dc_rs.plot()

In [ ]:
height = bg_dc_rs.range
temperature_prf = np.ones(height.size)*np.nan
pressure_prf = np.ones(height.size)*np.nan
for i, _height in enumerate(height):
    sa = helper_functions.standard_atmosphere(_height)
    pressure_prf[i] = sa[0]
    temperature_prf[i] = sa[1]

wavelength=532
molecular_profiles = lidarQA.molecular_properties(wavelength, pressure_prf, temperature_prf, height, component='total')
abetamol = molecular_profiles['attenuated_molecular_beta']    

In [ ]:
rcs = bg_dc_rs*bg_dc_rs.range**2
nrcs = rcs/rcs.sel(range=slice(5000,6000)).mean('range')
nabetamol = abetamol/abetamol.sel(range=slice(5000,6000)).mean('range')
nrcs.plot(x='range')
nabetamol.plot(x='range')
# plt.ylim(10,5e7)
plt.xlim(0,30000)
plt.yscale('log')

In [ ]:
reload(lidar)
path1a = r'C:\Users\Usuario\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_elastico\data\1a'

rs_fl = os.path.join(path1a, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Prs_rs_xf_%s.nc' % datestr)  
dc_fl = os.path.join(path1a, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Pdc_rs_xf_%s*.nc' % datestr)  
print(rs_fl,os.path.isfile(rs_fl))
#Lidar signal
ds = lidar.preprocessing(rs_fl, dc_fl=dc_fl,ini_date=ini_date_str, end_date=end_date_str) 

In [ ]:
rcs_532xpa = ds['signal_532xpa']*ds.range**2
rcs_532xpp = ds['signal_532xpp']*ds.range**2
rcs_532xsa = ds['signal_532xsa']*ds.range**2
rcs_532xsp = ds['signal_532xsp']*ds.range**2

In [ ]:
ds['signal_532xsa'].mean('time').plot(x='range',label='532xsa')
ds['signal_532xpa'].mean('time').plot(x='range',label='532xpa')
bg_dc_rs.plot(x='range')
plt.legend()
plt.ylim(-0.025,0.01)

In [ ]:
good = xr.open_dataset(r'Y:\datos\MULHACEN\QA\sent2ACTRIS\july2021\rayleighfit_20210624T223000\grRayleighFit532xpa.nc')


In [ ]:
good

In [ ]:
(good.RCS/(good.range)**2).plot()
plt.xlim(100,30000)
plt.ylim(0,0.1)

In [ ]:
good.DC.plot()
plt.xlim(100,30000)
plt.ylim(7.87,7.89)

In [ ]:
good['range'] = good.range*1000

In [ ]:
(ds['signal_532xpa'].mean('time')*ds.range**2).plot(x='range',label='532xpa-new')
good.RCS.plot(x='range',label='old')
plt.legend()


In [ ]:
rcs_532xpa_profile = rcs_532xpa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xpp_profile = rcs_532xpp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsa_profile = rcs_532xsa.sel(time=slice(ini_date_str,end_date_str)).mean('time')
rcs_532xsp_profile = rcs_532xsp.sel(time=slice(ini_date_str,end_date_str)).mean('time')
nrcs_532xpa = rcs_532xpa_profile/rcs_532xpa_profile.sel(range=slice(4000,5000)).mean('range')
nrcs_532xpp = rcs_532xpp_profile/rcs_532xpp_profile.sel(range=slice(4000,5000)).mean('range')
nrcs_532xsa = rcs_532xsa_profile/rcs_532xsa_profile.sel(range=slice(4000,5000)).mean('range')
nrcs_532xsp = rcs_532xsp_profile/rcs_532xsp_profile.sel(range=slice(4000,5000)).mean('range')



In [ ]:
lr_aer = 40
ymin = 4000
ymax = 5000

rcs_dict={532: {'ff': 'corrected_rcs_04', 'nf': 'corrected_rcs_30'},
          355: {'ff': 'corrected_rcs_12', 'nf': 'corrected_rcs_38'}, 
         1064: {'ff': 'corrected_rcs_00', 'nf': 'corrected_rcs_28'}}
channel_dict={532: {'ff':  4, 'nf': 30}, 
              355: {'ff': 12, 'nf': 38},
             1064: {'ff':  0, 'nf': 28}}


In [ ]:
iwds = ds.sel(time=slice(ini_date,end_date))
height = iwds.range
resolution = np.median(np.diff(height))
nanprf = np.ones(len(height))*np.nan
#Inversión KLETT para hallar la razón lidar
beta_part ={355: nanprf, 532: nanprf, 1064: nanprf}
beta_total ={355: nanprf, 532: nanprf, 1064: nanprf}
beta_mol ={355: nanprf, 532: nanprf, 1064: nanprf}
alpha_mol ={355: nanprf, 532: nanprf, 1064: nanprf}
od_total ={355: nanprf, 532: nanprf, 1064: nanprf}
att_beta_total ={355: nanprf, 532: nanprf, 1064: nanprf}
att_beta_total_rcs ={355: nanprf, 532: nanprf, 1064: nanprf}

In [ ]:
height = iwds.range
temperature_prf = np.ones(height.size)*np.nan
pressure_prf = np.ones(height.size)*np.nan
for i, _height in enumerate(height):
    sa = helper_functions.standard_atmosphere(_height)
    pressure_prf[i] = sa[0]
    temperature_prf[i] = sa[1]

wavelength=532
molecular_profiles = lidarQA.molecular_properties(wavelength, pressure_prf, temperature_prf, height, component='total')
abetamol = molecular_profiles['attenuated_molecular_beta']    

In [ ]:
nrcs_532xpa.plot(x='range', label='532xpa')
nrcs_532xpp.plot(x='range', label='532xpp', lw=4)
# nrcs_532xsa.plot(x='range', label='532xsa')
nrcs_532xsp.plot(x='range', label='532xsp',lw=1)
(abetamol/abetamol.sel(range=slice(4000,5000)).mean('range')).plot(x='range',label='mol',color='k')
plt.legend()
plt.savefig('rayleigh_utils-gfat.png')

In [ ]:

for channel_ in ['355xta', '532xta', '1064xta']: #[355, 532, 1064]  
    wavelength = int(channel_[0:-3])
    height = iwds.range.values 
    #Atmospheric conditions from 
    molecular_profiles = lidarQA.molecular_properties(wavelength, pressure_prf, temperature_prf, height, component='total')
    beta_mol[wavelength] = molecular_profiles['molecular_beta'].values    
    lr_mol = molecular_profiles['molecular_lidar_ratio'].values   
    alpha_mol[wavelength] = molecular_profiles['molecular_alpha'].values   
    abetamol = molecular_profiles['attenuated_molecular_beta']    
    rcs_ = iwds['signal_%s' % channel_].mean('time').values*height**2
    beta_part[wavelength] = lidar_elastic_retrieval.klett(rcs_, height, beta_mol[wavelength], lr_mol,
                                  lr_aer = 50, ymin = 4000, ymax = 5000)

    od_total[wavelength] = integrate.cumtrapz(alpha_mol[wavelength] + beta_part[wavelength]*lr_aer, dx=resolution, initial=0)
    beta_total[wavelength] = beta_part[wavelength]+beta_mol[wavelength]
    att_beta_total[wavelength] = beta_total[wavelength]*np.exp(-2*od_total[wavelength])

In [ ]:
fig, ax = plt.subplots(figsize=[15,10])
ax1 = plt.subplot(1,3,1)
ax1.plot(beta_mol[355], height, c='lightblue', lw=3, label='355mol')
ax1.plot(beta_total[355], height, c='b', label='355total')
ax1.plot(beta_mol[532], height, c='lightgreen', lw=3, label='532mol')
ax1.plot(beta_total[532], height, c='green', label='532total')
ax1.plot(beta_mol[1064], height, c='pink', lw=3, label='1064mol')
ax1.plot(beta_total[1064], height, c='red', label='1064total')
ax1.set_xlabel('$\\beta, [m^{-1}\cdot sr^{-1}]$')
ax1.set_ylabel('Range, [m agl]')
ax1.set_xlim(-5e-7,1.25e-5)
ax1.set_ylim([0,8000])
ax1.legend(fontsize='large')
ae355_532 = -np.log(beta_part[532]/beta_part[355])/np.log(532/355)
ae532_1064 = -np.log(beta_part[1064]/beta_part[532])/np.log(1064/532)
ae355_1064 = -np.log(beta_part[1064]/beta_part[355])/np.log(1064/355)
ax2 = plt.subplot(1,3,2)
ax2.plot(ae355_532, height, c='r', label='355-532')
ax2.plot(ae532_1064, height, c='b', label='532-1064')
ax2.plot(ae355_1064, height, c='g', label='355-1064')
ax2.set_xlim(-0.5,4)
ax2.set_xlabel('$Angs. exp., [#]$')
ax2.legend(fontsize='large')
ax2.set_ylim([0,8000])
ax2.set_yticklabels([])
ax3 = plt.subplot(1,3,3)
ax3.plot(att_beta_total[355], height, c='b', label='355')
ax3.plot(att_beta_total[532], height, c='g', label='532')
ax3.plot(att_beta_total[1064], height, c='r', label='1064')
# ax3.set_xlim(-0.5,4)
ax3.legend(fontsize='large')
ax3.set_xlabel('$\\beta_{att}, [m^{-1}\cdot sr^{-1}]$')
ax3.set_ylim([0,8000])
ax3.set_yticklabels([])
for ax_ in [ax1, ax2, ax3]:
    plot.font_axes(ax_)

In [ ]:
datestr, hour_ini, hour_end = '20210724', '000000', '003000' 
date_ini = dt.datetime.strptime(datestr,'%Y%m%d')
date_ = date_ini
year, month, day = date_.year, date_.month, date_.day
ini_date_str = '%sT%s' % (datestr, hour_ini)
end_date_str = '%sT%s' % (datestr, hour_end)
ini_date = dt.datetime.strptime(ini_date_str,'%Y%m%dT%H%M%S')
end_date = dt.datetime.strptime(end_date_str,'%Y%m%dT%H%M%S')

#Lidar signal
reload(lidar)
rs_fl = os.path.join(path1a_win, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Prs_rs_xf_%s.nc' % datestr)  
dc_fl = os.path.join(path1a_win, '%d' % year, '%02d' % month , '%02d' % day, 'mhc_1a_Pdc_rs_xf_%s*.nc' % datestr)  
print(rs_fl,os.path.isfile(rs_fl))

#Lidar signal
iwds = lidar.preprocessing(rs_fl, dc_fl=dc_fl, end_range=14000, percentage_required=50) 
height = iwds.range
resolution = np.median(np.diff(height))
nanprf = np.ones(len(height))*np.nan

In [ ]:
lr_aer = 45
ymin = 6500
ymax = 7500

#Meteo
ecmwf_ds = ecmwf.meteo_ecmwf(ini_date, ini_date, height)
temperature_prf = ecmwf_ds['temperature'].values
pressure_prf = ecmwf_ds['pressure'].values

if iwds:
    hours = np.arange(0,24,dtype='float')
    n_hour = len(hours)
    colors = mpl.pylab.cm.jet(np.linspace(0,1,n_hour))
#     fig, ax = plt.subplots(figsize=[5,7])
    for i, hour_ in enumerate(hours):        
        ini = date_ini + dt.timedelta(hours=hour_)
        end = date_ini + dt.timedelta(hours=hour_+1)
        inistr = ini.strftime('%Y%m%dT%H%M%S')
        endstr = end.strftime('%Y%m%dT%H%M%S')
        print(inistr,endstr)
        #Inversión KLETT para hallar la razón lidar
        beta_part ={355: nanprf, 532: nanprf, 1064: nanprf}
        beta_total ={355: nanprf, 532: nanprf, 1064: nanprf}
        beta_mol ={355: nanprf, 532: nanprf, 1064: nanprf}
        alpha_mol ={355: nanprf, 532: nanprf, 1064: nanprf}
        od_total ={355: nanprf, 532: nanprf, 1064: nanprf}
        att_beta_total ={355: nanprf, 532: nanprf, 1064: nanprf}
        att_beta_total_rcs ={355: nanprf, 532: nanprf, 1064: nanprf}
        for channel_ in ['355xta', '532xta', '1064xta']: #[355, 532, 1064]  
            print(channel_)
            wavelength = int(channel_[0:-3])
            height = iwds.range.values 
            #Atmospheric conditions from 
            molecular_profiles = lidarQA.molecular_properties(wavelength, pressure_prf, temperature_prf, height, component='total')
            beta_mol[wavelength] = molecular_profiles['molecular_beta'].values    
            lr_mol = molecular_profiles['molecular_lidar_ratio'].values   
            alpha_mol[wavelength] = molecular_profiles['molecular_alpha'].values   
            abetamol = molecular_profiles['attenuated_molecular_beta']    
            rcs_ = iwds['signal_%s' % channel_].sel(time=slice(inistr,endstr))
#             print(rcs_.time)
            process = len(rcs_.time)>0
            if process:
                rcs_ = rcs_.mean('time').rolling(range=5, center=True).mean().values*height**2
                beta_part[wavelength] = lidar_elastic_retrieval.klett(rcs_, height, beta_mol[wavelength], lr_mol,
                                              lr_aer = lr_aer, ymin = ymin , ymax = ymax)

                od_total[wavelength] = integrate.cumtrapz(alpha_mol[wavelength] + beta_part[wavelength]*lr_aer, dx=resolution, initial=0)
                beta_total[wavelength] = beta_part[wavelength]+beta_mol[wavelength]
                att_beta_total[wavelength] = beta_total[wavelength]*np.exp(-2*od_total[wavelength])                
        if  process:
            lidar_rcs = []
            lidar_rcs = xr.Dataset({
                         'backscatter355': (['range'], beta_part[355]),
                         'backscatter532': (['range'], beta_part[532]), 
                         'backscatter1064': (['range'], beta_part[1064]),
                         'molecular_backscatter355': (['range'], beta_mol[355]),
                         'molecular_backscatter532': (['range'], beta_mol[532]), 
                         'molecular_backscatter1064': (['range'], beta_mol[1064]),
                         'pressure_profile': (['range'], pressure_prf), 
                         'temperature_profile': (['range'], temperature_prf),
                         'particle_lidar_ratio': ([],lr_aer)},
                         coords={'range': iwds.range})
            lidar_rcs.to_netcdf(os.path.join(outputdir,'optical_properties_%s_%02d0000.nc' % (datestr, hour_)))
            fig,ax = plt.subplots()
            lidar_rcs['backscatter355'].plot(y='range',ax=ax,c='b',label='beta355')
            lidar_rcs['backscatter532'].plot(y='range',ax=ax,c='g',label='beta532')
            lidar_rcs['backscatter1064'].plot(y='range',ax=ax,c='r',label='beta1064')        
            ax.set_ylim(0,ymax)
            ax.set_xlim(0,10e-6)
            ax.legend()
            fig.savefig(os.path.join(outputdir,'optical_properties_%s_%02d0000.png' % (datestr, hour_)))
        else:
            print('No data in: %s-%s' % (inistr,endstr))
#         rcs2save[532].plot(y='range',ax=ax,c=colors[i],label='%02dh00' %hour_)
#         plot.font_axes(ax,14)
#         ax.set_title(datestr)
#         ax.legend(fontsize=8)
#         ax.set_xlabel('RCS@532nm, [a.u.]')
#         plt.tight_layout()
#         fig.savefig(os.path.join(outputdir,'rcs_%s.png' % datestr))
#         plt.close(fig)

In [ ]:
import xarray as xr

In [ ]:
fig,ax = plt.subplots(figsize=[15,10])
lidar_rcs['backscatter355'].plot(y='range',ax=ax,c='b',label='beta355')
ax.plot(beta_mol[355],height)
lidar_rcs['backscatter532'].plot(y='range',ax=ax,c='g',label='beta532')
lidar_rcs['backscatter1064'].plot(y='range',ax=ax,c='r',label='beta1064')        
ax.set_ylim(0,8000)
ax.legend()

In [ ]:
for key_ in iwds.keys():
    if key_.find('signal') != -1:
        print(key_)

In [ ]:
def function(input1, input2, inputA='hola', inputB='adiós'):
    
    
    salida = input1 + input2 + inputA + inputB
    
    
    return salida

In [ ]:
salida = function('input1', 'input2', inputA=' ')

In [ ]:
salida